# Importar librerias

In [5]:
import pandas as pd
import json
import os

# Extracción

Se carga y se cambia de formato el dataset crudo para que sea mas sencillo cargar y limpiar mas adelante.

## Metadata de hoteles

In [3]:
# Ruta de la carpeta de datos de Google Maps
google_maps_folder = '../raw_data/Google Maps/'

# Ruta de la subcarpeta "metadata-sitios"
google_maps_subfolder = os.path.join(google_maps_folder, 'metadata-sitios')

In [6]:
# Lista para almacenar los resultados filtrados
hotels = []

# Recorrer todos los archivos JSON dentro de "metadata-sitios"
for archivo in os.listdir(google_maps_subfolder):
    if archivo.endswith(".json"):
        # Ruta del archivo JSON actual
        ruta_archivo = os.path.join(google_maps_subfolder, archivo)

        # Leer el contenido del archivo JSON
        with open(ruta_archivo) as f:
            contenido_json = f.readlines()

        # Cargar cada objeto JSON por separado
        for linea in contenido_json:
            try:
                data = json.loads(linea)
            except json.JSONDecodeError as e:
                print(f"Error al cargar el archivo JSON {ruta_archivo}: {e}")
                continue

            # Verificar si la categoría "hotel" está presente en el archivo JSON
            categorias = data.get("category", [])
            if isinstance(categorias, list) and any("hotel" in categoria.lower() for categoria in categorias):
                hotels.append(data)

# Convertir los resultados a un pandas DataFrame
dfHotels = pd.DataFrame(hotels)

In [4]:
dfHotels.to_csv('../processed_data/hoteles_crudo.csv', index=False, sep='|')

In [5]:
dfHotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7700 entries, 0 to 7699
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              7700 non-null   object 
 1   address           7673 non-null   object 
 2   gmap_id           7700 non-null   object 
 3   description       1740 non-null   object 
 4   latitude          7700 non-null   float64
 5   longitude         7700 non-null   float64
 6   category          7700 non-null   object 
 7   avg_rating        7700 non-null   float64
 8   num_of_reviews    7700 non-null   int64  
 9   price             0 non-null      object 
 10  hours             36 non-null     object 
 11  MISC              665 non-null    object 
 12  state             37 non-null     object 
 13  relative_results  6058 non-null   object 
 14  url               7700 non-null   object 
dtypes: float64(3), int64(1), object(11)
memory usage: 902.5+ KB


In [6]:
dfHotels.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Basecamp Guesthouse,"Basecamp Guesthouse, 49010 SE Middle Fork Rd, ...",0x54907fca2751e187:0x3d6ae31ac18da483,None,47.471332,-121.686928,[Hotel],5.0,8,None,None,None,None,"[0x54907c35e89f333f:0x121a0cbfbbb39d3a, 0x549a...",https://www.google.com/maps/place//data=!4m2!3...
1,Sugar River Loft,"Sugar River Loft, 929 W Exchange St, Brodhead,...",0x88062b28cb53ef6d:0x94df72568fddf391,None,42.619862,-89.377890,[Hotel],4.7,8,None,None,None,None,"[0x88062b123701348f:0x5c12c6134b41325, 0x88062...",https://www.google.com/maps/place//data=!4m2!3...
2,Meraki Lodge,"Meraki Lodge, 223 Meraki Ln, Manson, WA 98831",0x549b9d4ad8f113d5:0x8258a10cca866a62,None,47.887652,-120.120376,"[Hotel, Group accommodation, Lodge, Motel]",4.9,15,None,None,None,None,"[0x549b9dd7e3512cd1:0x35541a27ae39f6df, 0x549b...",https://www.google.com/maps/place//data=!4m2!3...
3,View House at Lake Chelan,"View House at Lake Chelan, 245 Summerset Blvd,...",0x549b9db4d73626a3:0x348e8f7106d3be41,None,47.888291,-120.124120,[Hotel],4.4,8,None,None,None,None,"[0x549b9dd43b4f95f5:0x40d73fccf872ebe, 0x549b9...",https://www.google.com/maps/place//data=!4m2!3...
4,Station Lakefront Rental Home,"Station Lakefront Rental Home, 8188 Plum Lake ...",0x4d55cee0af102c25:0x752b313677a78e1c,None,45.998720,-89.490867,[Resort hotel],4.7,3,None,None,None,None,None,https://www.google.com/maps/place//data=!4m2!3...


In [7]:
print('Duplicados "hotels":', dfHotels['gmap_id'].duplicated().sum())

Duplicados "hotels": 63


In [7]:
dfHotels = dfHotels.drop_duplicates(subset='gmap_id') #63 observaciones

In [8]:
dfHotels.shape

(7637, 15)

# Reviews por estado

## Funcion para listar subcarpetas

In [2]:
def get_subfolders(folder_path):
    subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]
    return subfolders

folder_path_reviews_estados = "../raw_data/Google Maps/reviews-estados"
subfolders = get_subfolders(folder_path_reviews_estados)

In [11]:
subfolders

[]

## Funcion para unir archivos json

In [12]:
def read_json_files(folder_path):
    data = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".json"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                for line in file:
                    line = line.strip()
                    if line:
                        try:
                            json_obj = json.loads(line)
                            data.append(json_obj)
                        except json.JSONDecodeError:
                            continue

    df = pd.DataFrame(data)
    return df

## Funcion para unir los json en cada subcarpeta

Dependiendo de la capacidad de computo, es posible que se necesite realizar este paso con pocos archivos a la vez.

In [125]:
def read_json_files_in_subfolders(subfolders):
    df_list = []
    
    for subfolder in subfolders:
        df = read_json_files(subfolder)
        df_list.append(df)
    
    return df_list

dataframes_list = read_json_files_in_subfolders(subfolders)

Concatenar la lista en un df y guardar.

In [126]:
dfreviews = pd.concat(dataframes_list, ignore_index=True)

In [127]:
dfreviews.to_parquet('../raw_data/Google Maps/reviews-estados/reviews_google_crudo.parquet', index=False, compression="gzip")

En caso tal de haber dividido los archivos en varios parquet, se puede usar este codigo para concatenarlos y luego hacer merge con el dataframe de hoteles para reducir su tamaño.

In [11]:
def load_and_concat_parquet_files(folder_path):
    # Obtener una lista de todos los archivos Parquet en la carpeta
    parquet_files = [file for file in os.listdir(folder_path) if file.endswith('.parquet')]

    # Inicializar una lista para almacenar los DataFrames
    dfs = []

    # Cargar cada archivo Parquet en un DataFrame y agregarlo a la lista
    for file in parquet_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_parquet(file_path)
        dfs.append(df)

    # Concatenar los DataFrames en uno solo
    concatenated_df = pd.concat(dfs, ignore_index=True)

    return concatenated_df

folder_path = '../raw_data/Google Maps/reviews-estados/'
reviews_df = load_and_concat_parquet_files(folder_path)


## Uniendo hoteles con reviews

In [13]:
inner_merged = pd.merge(reviews_df, dfHotels, on=["gmap_id"])

In [16]:
inner_merged.to_csv("../raw_data/Google Maps/reviews-estados/reviews_hoteles_crudo.csv", index=False,  sep='|')